<a href="https://colab.research.google.com/github/dohyeonkim2526/BigContest/blob/master/BigCon0921.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [157]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [158]:
import os
import numpy as np
import pandas as pd

base_dir='/content/gdrive/My Drive/빅데이터 공모전/빅콘테스트 데이터/제공데이터'
sales_data='19년도-실적데이터.xlsx'
sales_dir=os.path.join(base_dir, sales_data)

base_dir_2='/content/gdrive/My Drive/빅데이터 공모전/빅콘테스트 데이터/평가데이터'
pred_data='20년6월-실적예측데이터.xlsx'
pred_dir=os.path.join(base_dir_2, pred_data)

# [예측 데이터]

In [3]:
df_preds=pd.read_excel(pred_dir,header=1)
df_preds  # 예측 상품 데이터프레임

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN
...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN


In [4]:
df_preds=df_preds.drop(df_preds[(df_preds['판매단가']==0)].index) # 예측 상품 판매가 0원인 것은 예측에서 제외
df_preds=df_preds.reset_index(drop=True)

In [5]:
# 예측 상품의 마더코드 리스트 생성

pred_code=[]
for code in df_preds['마더코드']:
  if code not in pred_code:
    pred_code.append(code)

pred_code.sort()

# [ 2019 판매 데이터 ]

In [159]:
df_sales=pd.read_excel(sales_dir,header=1, na_values='NaN')

In [263]:
del_idx = df_sales[df_sales['상품군']=='무형'].index
del_1 = df_sales.drop(del_idx)

In [188]:
del_1.isnull().sum()

방송일시              1
노출(분)         16785
마더코드              1
상품코드              1
상품명               1
상품군               1
판매단가              1
Unnamed: 7    37348
취급액            1994
dtype: int64

In [264]:
del_1['year']=del_1['방송일시'].dt.year
del_1['month']=del_1['방송일시'].dt.month
del_1['day']=del_1['방송일시'].dt.day_name()
del_1['hour']=del_1['방송일시'].dt.hour
del_1['min']=del_1['방송일시'].dt.minute

In [265]:
del_1=del_1.reset_index(drop=True)
del_1

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,Unnamed: 7,취급액,year,month,day,hour,min
0,2019-01-01 06:00:00,20.0,100346.0,201072.0,테이트 남성 셀린니트3종,의류,39900.0,NaN,2099000.0,2019.0,1.0,Tuesday,6.0,0.0
1,2019-01-01 06:00:00,NaN,100346.0,201079.0,테이트 여성 셀린니트3종,의류,39900.0,NaN,4371000.0,2019.0,1.0,Tuesday,6.0,0.0
2,2019-01-01 06:20:00,20.0,100346.0,201072.0,테이트 남성 셀린니트3종,의류,39900.0,NaN,3262000.0,2019.0,1.0,Tuesday,6.0,20.0
3,2019-01-01 06:20:00,NaN,100346.0,201079.0,테이트 여성 셀린니트3종,의류,39900.0,NaN,6955000.0,2019.0,1.0,Tuesday,6.0,20.0
4,2019-01-01 06:40:00,20.0,100346.0,201072.0,테이트 남성 셀린니트3종,의류,39900.0,NaN,6672000.0,2019.0,1.0,Tuesday,6.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37368,2020-01-01 00:00:00,20.0,100448.0,201383.0,무이자쿠첸압력밥솥 10인용,주방,178000.0,NaN,50929000.0,2020.0,1.0,Wednesday,0.0,0.0
37369,2020-01-01 00:00:00,NaN,100448.0,201390.0,일시불쿠첸압력밥솥 10인용,주방,168000.0,NaN,104392000.0,2020.0,1.0,Wednesday,0.0,0.0
37370,2020-01-01 00:00:00,NaN,100448.0,201384.0,무이자쿠첸압력밥솥 6인용,주방,158000.0,NaN,13765000.0,2020.0,1.0,Wednesday,0.0,0.0
37371,2020-01-01 00:00:00,NaN,100448.0,201391.0,일시불쿠첸압력밥솥 6인용,주방,148000.0,NaN,46608000.0,2020.0,1.0,Wednesday,0.0,0.0


In [266]:
sat = []
for i in range(del_1.shape[0]):
  row = del_1.iloc[i]
  row_day = row['day']
  row_hour = row['hour']
  row_min = row['min']
  if row_day == 'Saturday':
    if row_hour == 18:
      if row_min == 0 or row_min == 20:
        sat.append(i)
  else:
    continue

In [276]:
for i in range(del_1.shape[0]):
  if i in sat:
    del_1=del_1.drop(i,axis=0)

In [278]:
del_sat=del_1.reset_index(drop=True)

In [279]:
del_sat

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,Unnamed: 7,취급액,year,month,day,hour,min
0,2019-01-01 06:00:00,20.0,100346.0,201072.0,테이트 남성 셀린니트3종,의류,39900.0,NaN,2099000.0,2019.0,1.0,Tuesday,6.0,0.0
1,2019-01-01 06:00:00,NaN,100346.0,201079.0,테이트 여성 셀린니트3종,의류,39900.0,NaN,4371000.0,2019.0,1.0,Tuesday,6.0,0.0
2,2019-01-01 06:20:00,20.0,100346.0,201072.0,테이트 남성 셀린니트3종,의류,39900.0,NaN,3262000.0,2019.0,1.0,Tuesday,6.0,20.0
3,2019-01-01 06:20:00,NaN,100346.0,201079.0,테이트 여성 셀린니트3종,의류,39900.0,NaN,6955000.0,2019.0,1.0,Tuesday,6.0,20.0
4,2019-01-01 06:40:00,20.0,100346.0,201072.0,테이트 남성 셀린니트3종,의류,39900.0,NaN,6672000.0,2019.0,1.0,Tuesday,6.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37345,2020-01-01 00:00:00,20.0,100448.0,201383.0,무이자쿠첸압력밥솥 10인용,주방,178000.0,NaN,50929000.0,2020.0,1.0,Wednesday,0.0,0.0
37346,2020-01-01 00:00:00,NaN,100448.0,201390.0,일시불쿠첸압력밥솥 10인용,주방,168000.0,NaN,104392000.0,2020.0,1.0,Wednesday,0.0,0.0
37347,2020-01-01 00:00:00,NaN,100448.0,201384.0,무이자쿠첸압력밥솥 6인용,주방,158000.0,NaN,13765000.0,2020.0,1.0,Wednesday,0.0,0.0
37348,2020-01-01 00:00:00,NaN,100448.0,201391.0,일시불쿠첸압력밥솥 6인용,주방,148000.0,NaN,46608000.0,2020.0,1.0,Wednesday,0.0,0.0


In [298]:
test=[]

for i in range(del_sat.shape[0]):
  row=del_sat.iloc[i]
  row_day=row['day']
  row_hour=row['hour']
  if row_day == 'Saturday':
    if row_hour == 18:
      test.append(i)
  else:
    continue

In [299]:
del_test=[]

for day in test:
  del_sat_test=del_sat.iloc[day]
  del_test.append(del_sat_test)

testDF=pd.DataFrame(del_test)

In [300]:
testDF # 전체 데이터프레임에서 'Saturday' 18시 기준으로 (18:00 & 18:20) 데이터가 모두 지워졌음을 확인할 수 있다.

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,Unnamed: 7,취급액,year,month,day,hour,min
391,2019-01-05 18:40:00,20.0,100801.0,202365.0,바다먹자 국내산 손질꽃게 7팩,농수축,59900.0,NaN,50176000.0,2019.0,1.0,Saturday,18.0,40.0
1042,2019-01-12 18:40:00,20.0,100702.0,202074.0,영산포숙성 홍어8팩+무침3통,농수축,40900.0,NaN,61797000.0,2019.0,1.0,Saturday,18.0,40.0
1713,2019-01-19 18:40:00,20.0,100497.0,201498.0,이봉원 정직한 안창살 + 갈비살구이,농수축,59900.0,NaN,53751000.0,2019.0,1.0,Saturday,18.0,40.0
2395,2019-01-26 18:40:00,20.0,100199.0,200670.0,소들녘 소 갈비탕,농수축,50900.0,NaN,61505000.0,2019.0,1.0,Saturday,18.0,40.0
3017,2019-02-02 18:40:00,20.0,100841.0,202484.0,옛날 그 쥐포 110장 (11팩),농수축,32900.0,NaN,74412000.0,2019.0,2.0,Saturday,18.0,40.0
3746,2019-02-09 18:40:00,20.0,100321.0,201023.0,통째로 착즙한 루비 석류즙 84포,건강기능,77000.0,NaN,66508000.0,2019.0,2.0,Saturday,18.0,40.0
4446,2019-02-16 18:40:00,20.0,100651.0,201972.0,제주농장 유기 양배추진액 120포,건강기능,74900.0,NaN,69765000.0,2019.0,2.0,Saturday,18.0,40.0
5164,2019-02-23 18:40:00,20.0,100321.0,202185.0,통째로 착즙한 루비 석류즙 84포,건강기능,77000.0,NaN,67361000.0,2019.0,2.0,Saturday,18.0,40.0
5861,2019-03-02 18:40:00,20.0,100832.0,202444.0,청정수산 완도활전복 35미,농수축,59900.0,NaN,46942000.0,2019.0,3.0,Saturday,18.0,40.0
15272,2019-05-25 18:40:00,20.0,100322.0,201479.0,국내산 손질갑오징어 8팩,농수축,49900.0,NaN,69331000.0,2019.0,5.0,Saturday,18.0,40.0


In [301]:
# 2019 판매 상품의 마더코드 리스트 생성

sales_code=[]
for code in del_sat['마더코드']:
  if code not in sales_code:
    sales_code.append(code)

sales_code.sort()

## [2019 판매/예측 마더코드의 비교→예측 데이터에서 과거 실적이 없는 상품들을 찾아낸다.]

In [ ]:
no_sales_code=[]  # 예측 데이터 기준: 과거 실적이 없는 마더코드 리스트
for pred in pred_code:
  if pred not in sales_code:
    no_sales_code.append(pred)

no_sales_code.sort()

In [ ]:
df_only_in_preds=pd.DataFrame()

for code in no_sales_code:
  code_preds=df_preds[(df_preds['마더코드']==code)]
  df_only_in_preds=pd.concat([df_only_in_preds,code_preds])

df_only_in_preds=df_only_in_preds.reset_index(drop=True)

In [ ]:
df_only_in_preds  # 과거 실적이 없는 예측 상품 데이터프레임

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-06-25 08:20:00,20.0,100003,200004,PN 메가티타늄 후라이팬 기본세트,주방,68000,NaN
1,2020-06-25 08:20:00,NaN,100003,200003,PN 메가티타늄 후라이팬+IH 인덕션 1구 풀세트,주방,98000,NaN
2,2020-06-25 08:40:00,20.0,100003,200004,PN 메가티타늄 후라이팬 기본세트,주방,68000,NaN
3,2020-06-25 08:40:00,NaN,100003,200003,PN 메가티타늄 후라이팬+IH 인덕션 1구 풀세트,주방,98000,NaN
4,2020-06-25 09:00:00,20.0,100003,200004,PN 메가티타늄 후라이팬 기본세트,주방,68000,NaN
...,...,...,...,...,...,...,...,...
1192,2020-06-11 11:20:00,NaN,100847,202507,추억의쥐치포55장,농수축,32900,NaN
1193,2020-06-11 11:40:00,20.0,100847,202506,추억의쥐치포110장,농수축,32900,NaN
1194,2020-06-11 11:40:00,NaN,100847,202507,추억의쥐치포55장,농수축,32900,NaN
1195,2020-06-11 12:00:00,20.0,100847,202506,추억의쥐치포110장,농수축,32900,NaN




## [ 과거 실적이 없는 경우 분석 ]



* 예측 데이터 기준 과거 실적이 없는 경우 → 19년도 판매 데이터에 해당 제품의 마더 코드가 없음을 의미한다.

In [ ]:
df_only_in_preds

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-06-25 08:20:00,20.0,100003,200004,PN 메가티타늄 후라이팬 기본세트,주방,68000,NaN
1,2020-06-25 08:20:00,NaN,100003,200003,PN 메가티타늄 후라이팬+IH 인덕션 1구 풀세트,주방,98000,NaN
2,2020-06-25 08:40:00,20.0,100003,200004,PN 메가티타늄 후라이팬 기본세트,주방,68000,NaN
3,2020-06-25 08:40:00,NaN,100003,200003,PN 메가티타늄 후라이팬+IH 인덕션 1구 풀세트,주방,98000,NaN
4,2020-06-25 09:00:00,20.0,100003,200004,PN 메가티타늄 후라이팬 기본세트,주방,68000,NaN
...,...,...,...,...,...,...,...,...
1192,2020-06-11 11:20:00,NaN,100847,202507,추억의쥐치포55장,농수축,32900,NaN
1193,2020-06-11 11:40:00,20.0,100847,202506,추억의쥐치포110장,농수축,32900,NaN
1194,2020-06-11 11:40:00,NaN,100847,202507,추억의쥐치포55장,농수축,32900,NaN
1195,2020-06-11 12:00:00,20.0,100847,202506,추억의쥐치포110장,농수축,32900,NaN


In [ ]:
product_group=df_preds['상품군'].unique()
product_group

array(['의류', '속옷', '이미용', '주방', '농수축', '가전', '생활용품', '가구', '잡화', '건강기능',
       '침구'], dtype=object)

## [19년도 데이터 분석 - 판매량 기준]

* 주문량에 대한 new column 생성 + 취급액 0원인 것은 주문량인 0인 것이므로 제거

In [ ]:
def cal_sales(x,y):
  return round(x/y)

perform['주문량']=cal_sales(perform['취급액'],perform['판매단가'])
perform=perform.dropna(subset=['취급액'])
perform_1=perform.sort_values(by=['주문량'],ascending=False)

In [ ]:
perform_1

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,주문량
32442,2019-12-07 01:40:00,17.166667,100418,201322,코몽트 덤블 플리스 자켓 2종,의류,39900,322009000.0,8070.0
30072,2019-11-10 17:00:00,20.000000,100253,200864,안동간고등어 20팩,농수축,30900,199541000.0,6458.0
33871,2019-12-21 17:40:00,20.000000,100418,201322,코몽트 덤블 플리스 자켓 2종,의류,39900,223492000.0,5601.0
34288,2019-12-27 00:00:00,20.000000,100323,201025,피시원 국내산 햇 손질문어 7팩,농수축,49900,270284000.0,5417.0
31167,2019-11-22 19:00:00,20.000000,100317,201665,국내산 손질 햇 갑오징어 9팩,농수축,49900,241079000.0,4831.0
...,...,...,...,...,...,...,...,...,...
27053,2019-10-08 10:00:00,NaN,100787,202313,[루이띠에] 18k 인피니티 러브 팔찌,잡화,470000,663000.0,1.0
19034,2019-07-16 00:20:00,20.000000,100153,200558,일시불 LG전자 휴대용 퓨리케어 미니 공청기,가전,309000,409000.0,1.0
19036,2019-07-16 00:40:00,20.000000,100153,200558,일시불 LG전자 휴대용 퓨리케어 미니 공청기,가전,309000,421000.0,1.0
27052,2019-10-08 10:00:00,NaN,100787,202314,[루이띠에] 18k 인피니티러브 목걸이,잡화,530000,743000.0,1.0


In [ ]:
perform_1.groupby('상품군')['상품군'].count() #(19년도 판매량 순위) 주방 > 가전 > 의류 > 속옷 > 농수축> 잡화 > 가구 > 기타

상품군
가구      2268
가전      4783
건강기능     756
농수축     3484
생활용품    2571
속옷      3495
의류      3990
이미용     1237
잡화      3104
주방      6112
침구       649
Name: 상품군, dtype: int64

## [19년도 데이터 분석 - 방송시간 기준]




In [ ]:
perform_2=perform[perform['노출(분)'].notnull()]

In [ ]:
perform_2_NaN=perform[perform['노출(분)'].isnull()]
uni=perform_2_NaN['상품명'].unique()

In [ ]:
check_uni=[] # 한 상품명에 대하여 노출(분)이 0인 것에 대해 예외가 존재하는지 확인하기 위한 리스트 생성

for name in perform_2:
  if name in uni:
    check_uni.append(name)
  check_uni.sort()

check_uni

[]

* 노출(분)이 0인 상품의 경우, 노출(분)이 0이 아닌 예외적인 경우가 존재하지 않음을 알 수 있다.

In [ ]:
perform_2_NaN

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,주문량
4,2019-01-05 01:00:00,NaN,100293,200949,무이자[가이거] 제니스시계 주얼리세트,잡화,139000,6857000.0,49.0
6,2019-01-05 01:20:00,NaN,100293,200949,무이자[가이거] 제니스시계 주얼리세트,잡화,139000,15561000.0,112.0
8,2019-01-05 01:40:00,NaN,100293,200949,무이자[가이거] 제니스시계 주얼리세트,잡화,139000,8495000.0,61.0
13,2019-01-05 07:00:00,NaN,100222,200774,아문센 여성 윈터 하드쉘 세트,의류,79000,3811000.0,48.0
15,2019-01-05 07:20:00,NaN,100222,200774,아문센 여성 윈터 하드쉘 세트,의류,79000,8326000.0,105.0
...,...,...,...,...,...,...,...,...,...
34347,2019-12-27 18:20:00,NaN,100610,201906,푸마 코튼 언더탑 9종,속옷,89000,420000.0,5.0
34349,2019-12-27 18:40:00,NaN,100610,201884,푸마 시그니처 코튼 트렁크 9종,속옷,89000,5141000.0,58.0
34350,2019-12-27 18:40:00,NaN,100610,201906,푸마 코튼 언더탑 9종,속옷,89000,3634000.0,41.0
34352,2019-12-27 19:00:00,NaN,100610,201884,푸마 시그니처 코튼 트렁크 9종,속옷,89000,5102000.0,57.0


In [ ]:
count=perform_2_NaN.groupby('상품명')['주문량'].sum()
count=count.to_frame()
count

,주문량
상품명,
[가이거] 블랙 에디션 다이아몬드 워치(남성용),185.0
[맛있는 제주]손질 생선 3대세트+흑돼지구이(광어+갈치+고등어,461.0
메디아글램 카이만엠보 소가죽 체인숄더백 (+클러치백+폼폼이),728.0
메디아글램 카이만엠보 소가죽 토트백 (+클러치백+폼폼이),558.0
(1등급)221L_딤채김치냉장고,62.0
...,...
효재 자수 광목 워싱 침구세트 퀸,189.0
효재 자수 광목 워싱 침구세트 킹,373.0
효재 화원 자수 광목워싱 침구세트 슈퍼싱글,135.0
